# DevCloud API

Methods to perform GET/DELETE operations on the Queue Jobs.

In [2]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


In [5]:
import requests
import json
import liveQStat


def doGetJWT():
    JWT_URL = "https://nebula.udacity.com/api/v1/remote/intel-dev-cloud"
    TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
    token = requests.request("GET", TOKEN_URL, headers={"Metadata-Flavor":"Google"}).text
    jobStartJwt = requests.request(
        "POST", JWT_URL,
        headers={'Authorization': "Star " + token, "Content-Type": "application/json"},
        data=json.dumps({ 'nodeType' : 'GPU' })).text
    return json.loads(jobStartJwt)["jwt"]


class JWTAuth(requests.auth.AuthBase):
    def __init__(self, jwt):
        self.jwt = jwt
    def __call__(self, r):
        # Setting HTTP header for JWT Authorization
        r.headers['Authorization'] = "Bearer " + self.jwt
        return r


def deleteJob(job_id):
    jwt = doGetJWT()
    result = requests.delete('https://devapi.edge.devcloud.intel.com/jobapi/jobs/' + job_id, auth=JWTAuth(jwt))
    print(result.status_code, result.text)
    

def getJob(job_id):
    jwt = doGetJWT()
    result = requests.get('https://devapi.edge.devcloud.intel.com/jobapi/jobs/' + job_id, auth=JWTAuth(jwt))
    print(result.status_code, result.text)
    
    
def viewJobs():
    liveQStat.liveQStat()
    

## View Jobs in the Queue

In [7]:
viewJobs()

## Perform Operations on the Job

In [8]:
getJob('0BfwOOVz0Jf2dnBKwu2Qd3ug1bCaKAgD')

200 {"data":{"desc":"Job is completed, but waiting on results.","status":"W"},"status":"success"}

